In [6]:
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.viz.bokeh.MultiPlot import MultiPlot
from ndlib.viz.bokeh.DiffusionTrend import DiffusionTrend
import matplotlib.pyplot as plt

import networkx as nx
import pandas as pd
import json

from bokeh.io import output_notebook, show

input_graph_file = "data/vk_network.csv"
top_nodes_file = "data/top_nodes.json"
model_choice = "IC"
num_iterations = 100

beta = 0.05
gamma = 0.01
_lambda = 0.01
edge_threshold = 0.5
fraction_infected = 0.05

df = pd.read_csv(input_graph_file)
G = nx.from_pandas_edgelist(df, create_using=nx.DiGraph())

with open(top_nodes_file, "r") as f:
    infected_nodes = json.load(f)

print(f"Модель: {model_choice}")
print(f"Начально заражённые узлы: {infected_nodes}")

vm = MultiPlot()

if model_choice == "SIS":
    model = ep.SISModel(G)
    config = mc.Configuration()
    config.add_model_parameter("beta", beta)
    config.add_model_parameter("lambda", _lambda)
    config.add_model_parameter("fraction_infected", fraction_infected)
    config.add_model_initial_configuration("Infected", infected_nodes)
    model.set_initial_status(config)

elif model_choice == "SIR":
    model = ep.SIRModel(G)
    config = mc.Configuration()
    config.add_model_parameter("beta", beta)
    config.add_model_parameter("gamma", gamma)
    config.add_model_parameter("fraction_infected", fraction_infected)
    config.add_model_initial_configuration("Infected", infected_nodes)
    model.set_initial_status(config)

elif model_choice == "IC":
    model = ep.IndependentCascadesModel(G)
    config = mc.Configuration()
    config.add_model_initial_configuration("Infected", infected_nodes)
    for e in G.edges():
        config.add_edge_configuration("threshold", e, edge_threshold)
    model.set_initial_status(config)

else:
    raise ValueError("Неверный выбор модели. Используй SIS, SIR или IC.")

iterations = model.iteration_bunch(num_iterations)
print(model.get_info())

trends = model.build_trends(iterations)
p1 = DiffusionTrend(model, trends).plot(width=400, height=400)

vm.add_plot(p1)
m = vm.plot()

output_notebook()
show(m)



Модель: IC
Начально заражённые узлы: [65335269, 50923683, 70788979, 7819161, 61559640, 50889883, 62805638, 66431483, 580795050, 55439091, 75594886, 107398494, 171407397, 171962550, 173385410, 182450799, 185123946, 224310882, 227184764, 229688444, 245561026, 253682130, 254640574, 292095937, 354892031, 379883113, 393085904, 409069718, 424594649, 428384564, 444938235, 447513436, 470993772, 494676913, 690912992, 352758938, 143952, 446111, 694936, 942631, 1489031, 1575636, 1642443, 1783365, 2021235, 2250520, 3596409, 4554717, 4618828, 5270434, 5767138, 7553648, 8116023, 8189188, 8761746, 9140516, 9188399, 13349271, 14427992, 16482582, 16743398, 18479484, 19151353, 23971971, 25580148, 26633234, 27311015, 27353953, 28024216, 28269552, 29019774, 29164143, 29585408, 30205784, 32072659, 32632973, 33248561, 33581687, 36958372, 37612956, 38769537, 41378535, 42288224, 43615452, 46041007, 46769059, 49814723, 52447648, 56036322, 58181596, 65836069, 66342885, 68049427, 70185217, 73381220, 75514260, 77

Loading BokehJS ...